In [ ]:
import comtypes.client
import random

In [ ]:

# 连接到运行中的 SAP2000
sap_object = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
SapModel = sap_object.SapModel

# 解锁模型
SapModel.SetModelIsLocked(False)

# 设置单位
SapModel.SetPresentUnits(9)  # 设定单位为 N/mm^2

num_z = 3
num_x = 7
num_y = 7

SapModel.SetModelIsLocked(False)

In [ ]:
def random_goods_distribution(num_z, num_x, num_y):
    # 创建一个 num_z 行，num_x * num_y 列的矩阵
    distribution_matrix = []

    #创建一个num_z*num_x*num_y的一维数组
    good_id = []
    
    for layer in range(num_z):
        # 随机生成当前层的货物分布，0 或 1
        distribution = [random.choice([0, 1]) for _ in range(num_x * num_y)]
        distribution_matrix.append(distribution)  # 将当前层的分布添加到矩阵中
        
    return distribution_matrix

# 示例调用
num_z = 3  # 层数
num_x = 7  # 每层的货位数（假设）
num_y = 7  # 每层的货位数（假设）

distribution_matrix = random_goods_distribution(num_z, num_x, num_y)
for layer in distribution_matrix:
    print(layer)

print(distribution_matrix)

#转为一维数组
#flattened_distribution = [item for sublist in distribution_matrix for item in sublist]
#print(flattened_distribution)


In [ ]:
# 循环进行随机货位分布并计算频率
num_trials = 10 # 模拟次数
total_weight_per_layer = 1  # 每层1t重
surface_pressure_area = 1000000*9.8 / (2000*2000)  # 均布面力，可以根据需要调整


In [ ]:
#根据货位分布矩阵，对结构施加均布面力，然后进行模态分析
num_loc = num_x * num_y*num_z  # 货位总数
for trial in range(num_trials):
    # 随机分布货位
    distribution_matrix = random_goods_distribution(num_z, num_x, num_y)
    
    print(f"Trial {trial + 1}: Goods Distribution Matrix:")
    for layer in distribution_matrix:
        print(layer)
    
    flattened_distribution = [item for sublist in distribution_matrix for item in sublist]

    # 在 SAP2000 中添加均布面力
    for loc in range(num_loc):
        if flattened_distribution[loc] == 1:
            #todo：这里的函数名可能不对
            #SapModel.Loads.AddSurfaceLoad("Load Pattern Name", surface_index, "Uniform", surface_load)  # 在对应虚拟面添加均布面力
            sap_object.SapModel.AreaObj.SetLoadSurfacePressure("loc","DEAD",-2,surface_pressure_area," ",False,0)  # 设置均布面力大小

    #执行模态分析
    SapModel.Analyze.RunAnalysis()
    ret = SapModel.Analyze.RunAnalysis()
    if ret != 0:
        print("Analysis did not run successfully.")
    else:
        print("Analysis run successfully.")

    NumberResults = 8
    LoadCase = []
    StepType = []
    StepNum = []
    Period = []
    Frequency = []
    CircFreq = []
    EigenValue = []

    # 调用函数获取模态数据
    result = SapModel.Results.ModalPeriod(NumberResults, LoadCase, StepType, StepNum, Period, Frequency, CircFreq, EigenValue)
print(result[6])  # 6 代表CircFreq

# 关闭 SAP2000
#sap_object.ApplicationExit(False)

SyntaxError: invalid syntax (1104171777.py, line 38)